# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)


Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
#reading csv files

train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [49]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = np.arange(0,30,2)
    
    #To ensure continuos flow of data across epochs
    while True:        
        #Shuffling the .csv files so that we get a random order of data and so the network wont be biased.
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        #considering the left out sequences
        if(len(t)%batch_size != 0):
            num_batches = num_batches+1
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                if (folder + (batch*batch_size)) > len(t)-1:
                    continue
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=resize(image,(100,100))
                    image=image/255
                    batch_data[folder,idx,:,:,0] =image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

               

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [50]:
#Calculating training sequences and defining batch size and epochs

curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =40 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size=64 # choose the batch size
print ('# batch size =', batch_size)


# training sequences = 663
# validation sequences = 100
# epochs = 40
# batch size = 64


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### Conv3D Model 
* Batch Size=64
*  No of Epochs=40

In [51]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

#write your model here
model = Sequential()
        
model.add(Conv3D(16, (3,3,3), padding='same',input_shape = (15,100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (3,3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5,activation='softmax'))
 


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [52]:
#Definibg optimiser and compiling the model and printing model summary

optimiser = optimizers.Adam()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 15, 100, 100, 16)  1312      
_________________________________________________________________
activation_37 (Activation)   (None, 15, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 15, 100, 100, 16)  64        
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 15, 100, 100, 32)  13856     
_________________________________________________________________
activation_38 (Activation)   (None, 15, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 15, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_34 (MaxPooling (None, 7, 50, 50, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [53]:
#Calling generator functions
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [54]:
#Processing steps

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [55]:
#Calculating stpes per epoch

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [56]:
#Fitting the model and saving 

history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 64
Source path =  ./Project_data/train ; batch size = 64
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


11/11 [==============================] - 84s 8s/step - loss: 1.8409 - categorical_accuracy: 0.2969 - val_loss: 2.0043 - val_categorical_accuracy: 0.2266

Epoch 00001: val_loss improved from inf to 2.00426, saving model to model_init_2020-02-0810_06_23.995411/model-00001-1.84090-0.29688-2.00426-0.22656.h5
Epoch 2/40
11/11 [==============================] - 53s 5s/step - loss: 1.3722 - categorical_accuracy: 0.4361 - val_loss: 2.0252 - val_categorical_accuracy: 0.2500

Epoch 00002: val_loss did not improve from 2.00426
Epoch 3/40
11/11 [==============================] - 58s 5s/step - loss: 1.1384 - categorical_accuracy: 0.5185 - val_loss: 1.8369 - val_categorical_accuracy: 0.2578

Epoch 00003: val_loss improved from 2.00426 to 1.83686, saving model to model_init_2020-02-0810_06_23.995411/model-00003-1.13839-0.51847-1.83686-0.25781.h5
Epoch 4/40
11/11 [==============================] - 58s 5s/step - loss: 0.9936 - categorical_accuracy: 0.5852 - val_loss: 0.9656 - val_categorical_accuracy: 


Epoch 00032: val_loss improved from 0.40716 to 0.39919, saving model to model_init_2020-02-0810_06_23.995411/model-00032-0.14237-0.92756-0.39919-0.87500.h5
Epoch 33/40
11/11 [==============================] - 59s 5s/step - loss: 0.1233 - categorical_accuracy: 0.9290 - val_loss: 0.3933 - val_categorical_accuracy: 0.8750

Epoch 00033: val_loss improved from 0.39919 to 0.39329, saving model to model_init_2020-02-0810_06_23.995411/model-00033-0.12334-0.92898-0.39329-0.87500.h5
Epoch 34/40
11/11 [==============================] - 59s 5s/step - loss: 0.1512 - categorical_accuracy: 0.9276 - val_loss: 0.3885 - val_categorical_accuracy: 0.8750

Epoch 00034: val_loss improved from 0.39329 to 0.38849, saving model to model_init_2020-02-0810_06_23.995411/model-00034-0.15116-0.92756-0.38849-0.87500.h5
Epoch 35/40
11/11 [==============================] - 59s 5s/step - loss: 0.1380 - categorical_accuracy: 0.9361 - val_loss: 0.3818 - val_categorical_accuracy: 0.8750

Epoch 00035: val_loss improved fr